## ICESat-2 SlideRule Download

This code is adapted from Sliderule Python example code. It is designed to calculate ATL06 elevations from ALT03 points and save those elevations as a CSV. 

Activate IS2alpinesnow enviroment to run

Last updated by Karina Zikan September 2022

In [ ]:
import sys
import logging
import concurrent.futures
import time
from datetime import datetime
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Transformer, CRS
from shapely.geometry import Polygon, Point
from sliderule import sliderule, icesat2, earthdata, h5

## SlideRule Configuration

In [ ]:
# Configure ICESat-2 API
icesat2.init("slideruleearth.io", verbose=False)

## Specify Region of Interest

#### 1. Make geojson from shapefile - if you have a geojson of the region skip to 2.

In [ ]:
# Specify region of interest shapefile
shp = "Sites/DCEW/ROIs/DCEW_border_WGS84.shp"
shapefile = gpd.read_file(shp)
# shapefile.to_file('DryCreek/boundDCEW/boundDCEW_WGS84.geojson', driver='GeoJSON')
shapefile.crs is None

##### - Check shapefile crs

In [ ]:
## if shapefile.crs is None is false check the crs
shapefile.crs

##### - Make Geojson

In [ ]:
shapefile.to_file('Sites/DCEW/ROIs/DCEW_border_WGS84.geojson', driver='GeoJSON')

In [ ]:
## if shapefile.crs is None is true run this section
#crs = 'epsg:8826' # set the starting datum using the epsg code
#icesat_crs = 4979 # epsg code for WGS84
#shapefile = shapefile.set_crs(crs)
#shapefile = shapefile.to_crs(icesat_crs) # transform to WGS84 
#shapefile.to_file('/Mores/MCSboundary/MoresBound_WGS84.geojson', driver='GeoJSON')

In [ ]:
## if shapefile.crs is not WGS84 run this section
#icesat_crs = 4979 # epsg code for WGS84
#shapefile = shapefile.to_crs(icesat_crs) # transform to WGS84 
#shapefile.to_file('/Mores/MCSboundary/MoresBound_WGS84.geojson', driver='GeoJSON')

#### 2. Read in geojson for Region of Interest

In [ ]:
# Specify region of interest from geojson
poly_fn = 'Sites/DCEW/ROIs/DCEW_border_WGS84.geojson'
region = icesat2.toregion(poly_fn)["poly"] # symplify 
region

In [ ]:
# Prepare coordinate lists for plotting the region of interest polygon
region_lon = [e["lon"] for e in region]
region_lat = [e["lat"] for e in region]

In [ ]:
# Read geojson with geopandas
pregion = gpd.read_file(poly_fn)
map = pregion.plot(); # plot origonal boundery 
map.plot(region_lon, region_lat, linewidth=1, color='g'); # plot symplified region boundery for a sanity check

#### 3. Save simplified polygon

In [ ]:
polygon_geom = Polygon(zip(region_lon, region_lat))
crs = 'epsg:4979'
polygon = gpd.GeoDataFrame(index=[0], geometry=[polygon_geom])       

polygon

In [ ]:
polygon.to_file('DryCreek/boundDCEW/boundDCEW_WGS84_simple.geojson', driver='GeoJSON')

## Specify parameters for ATL06-SR (Sliderule) processing

In [ ]:
# Build ATL06 Request #if remove "atl08_class", change icesat2.CNF_BACKGROUND to icesat2.CNF_SURFACE_HIGH
parms = {
    "poly": region,
    "srt": icesat2.SRT_LAND,
    "atl08_class": "atl08_ground",
    "cnf": icesat2.CNF_BACKGROUND, 
    "ats": 20.0,
    "cnt": 10,
    "len": 40.0,
    "res": 20.0,
    "maxi": 6
}
#    "atl08_class": "atl08_ground",
#    "cnf": icesat2.CNF_BACKGROUND,

#    "cnf": icesat2.CNF_SURFACE_HIGH,

cvs_name = "Mores/MC-ICESat2-ATL06sr-atl08class.csv" #set name of the output

## Calculate ATL06-SR Elevations from ATL03 Photons using SlideRule

In [ ]:
# Latch Start Time
perf_start = time.perf_counter()

# Request ATL06 Data
atl06_sr = icesat2.atl06p(parms)

# Latch Stop Time
perf_stop = time.perf_counter()

# Display Statistics
perf_duration = perf_stop - perf_start
print("Completed in {:.3f} seconds of wall-clock time".format(perf_duration))
#print("Reference Ground Tracks: {}".format(atl06_sr["rgt"].unique()))
#print("Cycles: {}".format(atl06_sr["cycle"].unique()))
print("Received {} elevations".format(atl06_sr.shape[0]))

## Plot ATL06-SR Results

In [ ]:
f, ax = plt.subplots()
ax.set_title("ATL06-SR Points")
ax.set_aspect('equal')
atl06_sr.plot(ax=ax, column='h_mean', cmap='inferno', s=0.1)
ax.plot(region_lon, region_lat, linewidth=1, color='g');

In [ ]:
#Set utmzone code - utm zone 11:32611
utm_code = 'epsg:32611'

atl06_sr['Longitude'] = atl06_sr['geometry'].x
atl06_sr['Latitude'] = atl06_sr['geometry'].y
eastnorths = atl06_sr['geometry'].to_crs(utm_code)
atl06_sr['Easting'] = eastnorths.x
atl06_sr['Northing'] = eastnorths.y
atl06_sr

## Write CSV

In [ ]:
atl06_sr.to_csv(cvs_name)